In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from random import randint 
from tqdm.notebook import trange, tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from random import sample

import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('../modules/')
from get_data import get_data
from get_gpt_reviews import get_gpt_reviews

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print(f"Device used = {device}")

Device used = cuda


In [2]:
!set PYTHONIOENCODING="latin1"

In [3]:
X_mov = pd.read_csv("../Data/moviedata.csv",delimiter=";",header=None, names=["sent","text"])
X_mov.dropna(inplace=True)
Y_mov = X_mov["sent"]
X_mov = X_mov["text"]
X,Y = X_mov,Y_mov

In [39]:
X, Y = get_data(type="gpt_2000", early_return=False)

In [25]:
X, Y = get_data()
samples = sample(list(zip(X,Y)),1600)
X = [k[0] for k in samples]
Y = [k[1] for k in samples]

In [40]:
vectorizer = TfidfVectorizer(min_df = 25, max_df = 0.8)
vectorizer.fit(X)
vocab = vectorizer.vocabulary_
vocab = {key:idx+1 for idx,key in enumerate(vocab)}
vocab["<PAD>"] = 0

#abtracting from actual words to just numbers
#cutoff length for sentences
sentence_length = 32

def normalize(data):
    word2idx = []
    for line in data:
        line = line.split()
        ans = [vocab.get(line[index], vocab["<PAD>"]) for index in range(min(sentence_length, len(line)))]
        for i in range(sentence_length - len(ans)):
            ans.append(vocab["<PAD>"])
        word2idx.append(ans)
    return word2idx

print("Length of vocab:",len(vocab))

X = normalize(X)

Length of vocab: 1103


In [41]:
lstm_dim = 100
embed_dim = 100

class LangID(nn.Module):
    def __init__(self, embed_dim, lstm_dim, vocab_dim):
        super(LangID, self).__init__()
        self.embedding = nn.Embedding(vocab_dim, embed_dim) #id, 100
        self.lstm = nn.LSTM(embed_dim,lstm_dim,batch_first = True, bidirectional = True)
        self.hidden2tag = nn.Linear(2*lstm_dim, 2)
        self.dropoutlayer = nn.Dropout(0.2)
    
    def forward(self, inputs):

        embeds = self.embedding(inputs)
        #print("embeds",embeds.shape)

        lstm_out, _ = self.lstm(self.dropoutlayer(embeds))
        #print("lstm_out",lstm_out.shape)
      
        tag_space = self.hidden2tag(self.dropoutlayer(lstm_out))[:,-1,:]
        #print("tag_space", tag_space.shape)
        return tag_space

In [42]:
source = torch.tensor(X)
target = torch.tensor(Y)

tmp_feats = source
tmp_labels = target
batch_size = 64
num_batches = int(len(tmp_labels)/batch_size)

tmp_feats_batches = tmp_feats[:batch_size*num_batches].view(num_batches,batch_size, sentence_length)
tmp_labels_batches = tmp_labels[:batch_size*num_batches].view(num_batches, batch_size)
tmp_feats_batches = tmp_feats_batches.to(device)
tmp_labels_batches = tmp_labels_batches.to(device)
#creating the model
model = LangID(embed_dim, lstm_dim, len(vocab))
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)
t = trange(15, desc='Started Training', leave=True, position=0)

for epoch in t:
    totalloss = 0
    
    for i in tqdm(range(len(tmp_feats_batches)), desc=f'Epoch {epoch+1} progress', leave=False, position=0):
    
        feats_batch = tmp_feats_batches[i]
        labels_batch = tmp_labels_batches[i]
        #print(feats_batch.shape, labels_batch.shape)
        # Here you can call forward/calculate the loss etc.
        model.zero_grad()
        tag_scores = model.forward(feats_batch)

        #print(tag_scores.shape)
        loss = loss_function(tag_scores, labels_batch)
        totalloss += loss.item()
        loss.backward()
        optimizer.step()
#         t2.set_description(f"Epoch {epoch+1} batch:{i}")
#         t2.refresh()

    t.set_description(f"Epoch {epoch+1} loss:{totalloss}")
    t.refresh()



Started Training:   0%|          | 0/15 [00:00<?, ?it/s]

Epoch 1 progress:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch 2 progress:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch 3 progress:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch 4 progress:   0%|          | 0/188 [00:00<?, ?it/s]

...

Epoch 11 progress:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch 12 progress:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch 13 progress:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch 14 progress:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch 15 progress:   0%|          | 0/188 [00:00<?, ?it/s]

In [43]:
Xt, Yt = get_data(type="dev")
Xt = normalize(Xt)
Xt = torch.tensor(Xt)
Yt = torch.tensor(Yt)
Xt = Xt.to(device)
Yt = Yt.to(device)

In [44]:
model.eval()
preds = torch.argmax(model.forward(Xt), dim = 1)
sum(preds == Yt)/len(Yt)

tensor(0.6958, device='cuda:0')

In [23]:
Xgpt, Ygpt = get_data(type="gpt_2000")
Xgpt = Xgpt["reviewText"]
Xgpt = list(Xgpt)
Ygpt = list(Ygpt)
Xgpt = normalize(Xgpt)
Xgpt = torch.tensor(Xgpt)
Ygpt = torch.tensor(Ygpt)
Xgpt = Xgpt.to(device)
Ygpt = Ygpt.to(device)

                                              reviewText  sentiment
0      leann rimes is one of my favorite artists to b...          1
1      these songs take me way back and i feel like i...          1
2      the worst of billy joel. he should stick to st...          0
3      according to frontman/songwriter ra mcguire, t...          0
4      tot is a great album, but it took forever for ...          0
...                                                  ...        ...
11161  i have listened to this album over and over  a...          1
11162  this is one of my favorites.  i can't wait to ...          1
11163  i like john rich's music and he has brought so...          1
11164  this album is so good.  i love the whole album...          1
11165  i first heard john rich on the radio and i had...          1

[11166 rows x 2 columns]


tensor(0.7147, device='cuda:0')

In [36]:
pos = (preds == Ytest)
pos = [bool(k) for k in pos]

In [34]:
Xraw, Yraw = get_data(type="dev")
Xraw = Xraw["reviewText"]

In [46]:
print(list(zip(Xraw[pos], Yraw[pos], [int(k) for k in preds[pos]])))

[('My dentist recommended this as a relaxation technique for dental visits. They give me an ipod with headphones, play this on it and it relieves some of the stress of dental treatment, which I dislike intensely.\nIt worked so well that I bought my own copy to try at home. I fall asleep after a couple of minutes and stay asleep. Instead of tossing and turning, I hardly move at all. Highly recommend.', 1, 1), ('I am personally acquainted with a member of this group and I love his music, but unfortunately, despite his efforts and talent which are not well exhibited herein, are for naught.  The lyrics are often unintelligible, the melodies are boring and repetitious, and the recording quality is marginal.  I am sad because I know these young folks are greatly inspired but it is not demonstrated here.', 0, 0), ('The Cd cover was broken when I got it', 0, 0), ("This is an uplifting, keep going, motivating song.  Because the Lord is truly  Big and strong . He is my everything. And I like to 

In [43]:
preds[pos]

tensor([1, 0, 0,  ..., 1, 0, 1], device='cuda:0')

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
